In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [ ]:
import gc
import torch

# Delete all references to GPU tensors

del model
del optimizer
# etc.

gc.collect()                  # Python garbage collection
torch.cuda.empty_cache()      # Free up unused memory
torch.cuda.ipc_collect()      # Collect IPC resources


In [ ]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset(
    "TarnishedNathe/islamic-QCM",
    split="train"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/691 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

data/dev-00000-of-00001.parquet:   0%|          | 0.00/565k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
datasetVal = load_dataset(
    "TarnishedNathe/islamic-QCM",
    split="dev"
)


In [ ]:

dataset

Dataset({
    features: ['id', 'question', 'A', 'B', 'C', 'D', 'answer', 'context_positive', 'context_negative', 'context_neutral'],
    num_rows: 600
})

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import math


In [ ]:
import random

class CustomDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=256):
        self.data = []
        self.tokenizer = tokenizer
        self.samples = []
        self.max_length = max_length

        for row in dataset:
            q, a = row['question'], row[row['answer']]
            pos = row['context_positive']
            neu = row['context_neutral']
            neg = row['context_negative']

            self.samples.append({
                "reg_text": f"{q} [SEP] {pos} [SEP] {a}",
                "anchor_text": f"{q} [SEP] {a}",
                "pos_text":    pos,
                "neg_text":    neu,
                "score": 1.0,
                "use_triplet": True
            })


            self.samples.append({
                "reg_text": f"{q} [SEP] {neu} [SEP] {a}",
                "anchor_text": f"{q} [SEP] {a}",
                "pos_text":    neu,
                "neg_text":    neg,
                "score": 0.7,
                "use_triplet": False # signals to skip triplet loss
            })

            self.samples.append({
                "reg_text": f"{q} [SEP] {neg} [SEP] {a}",
                "anchor_text": f"{q} [SEP] {neg} [SEP] {a}",
                "pos_text": f"{q} [SEP] {neg} [SEP] {a}",
                "neg_text": f"{q} [SEP] {neg} [SEP] {a}",
                "score": 0.0,
                "use_triplet": False  # signals to skip triplet loss
            })

            other_questions = [item['question'] for item in dataset if item['question'] != q]
            if other_questions:
                random_q = random.choice(other_questions)

            self.samples.append({
                "reg_text": f"{random_q} [SEP] {pos} [SEP] {a}",  # substitute q with random question
                "anchor_text": f"{q} [SEP] {neg} [SEP] {a}",
                "pos_text": f"{q} [SEP] {neg} [SEP] {a}",
                "neg_text": f"{q} [SEP] {neg} [SEP] {a}",
                "score": 0.7,
                "use_triplet": False  # signals to skip triplet loss
            })

            for choice in ['A', 'B', 'C', 'D']:
              if choice == row["answer"]:
                continue
              else:
                a = row[choice]
                self.samples.append({
                    "reg_text": f"{q} [SEP] {neg} [SEP] {a}",
                    "anchor_text": f"{q} [SEP] {neg} [SEP] {a}",
                    "pos_text": f"{q} [SEP] {neg} [SEP] {a}",
                    "neg_text": f"{q} [SEP] {neg} [SEP] {a}",
                    "score": 0.0,
                    "use_triplet": False  # signals to skip triplet loss
                })



    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        max_len = self.max_length

        reg_anchor = self.tokenizer(sample["reg_text"], truncation=True, padding='max_length', max_length=max_len, return_tensors='pt')
        tok_anchor = self.tokenizer(sample["anchor_text"], truncation=True, padding='max_length', max_length=max_len, return_tensors='pt')
        tok_pos    = self.tokenizer(sample["pos_text"], truncation=True, padding='max_length', max_length=max_len, return_tensors='pt')
        tok_neg    = self.tokenizer(sample["neg_text"], truncation=True, padding='max_length', max_length=max_len, return_tensors='pt')

        return {
            'reg_input_ids': reg_anchor['input_ids'].squeeze(0),
            'reg_attention_mask': reg_anchor['attention_mask'].squeeze(0),
            'anchor_input_ids': tok_anchor['input_ids'].squeeze(0),
            'anchor_attention_mask': tok_anchor['attention_mask'].squeeze(0),
            'pos_input_ids': tok_pos['input_ids'].squeeze(0),
            'pos_attention_mask': tok_pos['attention_mask'].squeeze(0),
            'neg_input_ids': tok_neg['input_ids'].squeeze(0),
            'neg_attention_mask': tok_neg['attention_mask'].squeeze(0),
            'score': torch.tensor(sample['score'], dtype=torch.float),
            'use_triplet': sample['use_triplet']
        }



In [ ]:
class RerankerModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        self.reg_head = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()  # ∈ [0, 1]
        )

    def encode(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pool = out.last_hidden_state[:, 0]  # CLS
        return pool

    def forward(self, reg_inputs, anchor_inputs=None, pos_inputs=None, neg_inputs=None):
        reg = self.encode(reg_inputs['input_ids'], reg_inputs['attention_mask'])
        score_pred = self.reg_head(reg).squeeze(-1)  # Regression output

        if anchor_inputs is not None and pos_inputs is not None and neg_inputs is not None:
            anchor = self.encode(anchor_inputs['input_ids'], anchor_inputs['attention_mask'])
            pos = self.encode(pos_inputs['input_ids'], pos_inputs['attention_mask'])
            neg = self.encode(neg_inputs['input_ids'], neg_inputs['attention_mask'])
            return anchor, pos, neg, score_pred
        else:
            return None, None, None, score_pred

In [ ]:
def compute_accuracy(preds, targets, threshold=0.5):
    correct = (torch.abs(preds - targets) < threshold).sum().item()
    return correct / targets.size(0)


In [ ]:
def validate(model, val_loader, device, threshold=0.1):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch in val_loader:
            reg_input = {
                "input_ids": batch['reg_input_ids'].to(device),
                "attention_mask": batch['reg_attention_mask'].to(device)
            }
            anchor_input = {
                "input_ids": batch['anchor_input_ids'].to(device),
                "attention_mask": batch['anchor_attention_mask'].to(device)
            }
            pos_input = {
                "input_ids": batch['pos_input_ids'].to(device),
                "attention_mask": batch['pos_attention_mask'].to(device)
            }
            neg_input = {
                "input_ids": batch['neg_input_ids'].to(device),
                "attention_mask": batch['neg_attention_mask'].to(device)
            }

            target_score = batch['score'].to(device)
            _, _, _, pred_score = model(reg_input, anchor_input, pos_input, neg_input)

            batch_correct = (torch.abs(pred_score - target_score) < threshold).sum().item()
            correct += batch_correct
            total += target_score.size(0)

    model.train()
    return correct / total if total > 0 else 0.0


In [ ]:

def train(model, dataloader, val_loader, optimizer, epochs=EPOCHS, alpha=0.7, accuracy_threshold=0.1, validate_every=200):
    device = "cuda"
    model = model.to(device)
    model.train()

    triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
        distance_function=lambda x, y: 1 - nn.functional.cosine_similarity(x, y),
        margin=TRIPLET_MARGIN,
        reduction='mean'
    )
    reg_loss_fn = nn.BCELoss()
    best_val_acc = 0.0
    step = 0
    exit = 0

    for epoch in range(epochs):
        total_loss = 0.0
        pbar = tqdm(dataloader, desc=f"Training Epoch {epoch+1}")
        for batch in pbar:
            step += 1
            reg_input = {
                "input_ids": batch['reg_input_ids'].to(device),
                "attention_mask": batch['reg_attention_mask'].to(device)
            }
            anchor_input = {
                "input_ids": batch['anchor_input_ids'].to(device),
                "attention_mask": batch['anchor_attention_mask'].to(device)
            }
            pos_input = {
                "input_ids": batch['pos_input_ids'].to(device),
                "attention_mask": batch['pos_attention_mask'].to(device)
            }
            neg_input = {
                "input_ids": batch['neg_input_ids'].to(device),
                "attention_mask": batch['neg_attention_mask'].to(device)
            }

            target_score = batch['score'].to(device)
            use_triplet = batch['use_triplet']

            optimizer.zero_grad()

            # Forward pass
            anchor, pos, neg, pred_score = model(reg_input, anchor_input, pos_input, neg_input)

            # Compute regression loss (always used)
            loss_reg = reg_loss_fn(pred_score, target_score)

            if use_triplet.any() and epoch > 2:
                valid_indices = use_triplet.nonzero(as_tuple=True)[0]
                loss_triplet = triplet_loss_fn(anchor[valid_indices], pos[valid_indices], neg[valid_indices])
                loss = alpha * loss_triplet + (1 - alpha) * loss_reg
            else:
                loss = loss_reg

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': total_loss / (pbar.n + 1)})

            if step % validate_every == 0:
                val_acc = validate(model, val_loader, device, threshold=accuracy_threshold)
                print(f"Step {step}: Validation Accuracy: {val_acc:.4f}")
                exit = exit + 1
                if val_acc > best_val_acc:
                    exit = 0
                    best_val_acc = val_acc
                    torch.save(model.state_dict(), 'best_model.pth')
                    print(f"Best model saved at step {step} with accuracy {val_acc:.4f}!")

        if exit == 5:
            print(f"early exiting at step {step} with accuracy {best_val_acc:.4f}!")



In [ ]:
dataset

Dataset({
    features: ['id', 'question', 'A', 'B', 'C', 'D', 'answer', 'context_positive', 'context_negative', 'context_neutral'],
    num_rows: 600
})

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

MODEL_NAME = "oddadmix/arabic-reranker"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
dataset = CustomDataset(dataset, tokenizer)
datasetVal = CustomDataset(datasetVal, tokenizer)

loader = DataLoader(dataset, batch_size=8, shuffle=True)
val = DataLoader(datasetVal, batch_size=8, shuffle=True)
# Init model + optimizer
model = RerankerModel(MODEL_NAME)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Train the model
train(model, loader, val, optimizer, epochs=20, alpha=0.2)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

Training Epoch 8:  24%|██▍       | 125/525 [04:27<4:13:37, 38.04s/it, loss=0.164]

Step 3800: Validation Accuracy: 0.9371


Training Epoch 8:  62%|██████▏   | 325/525 [10:19<2:07:05, 38.13s/it, loss=0.171]

Step 4000: Validation Accuracy: 0.9090


Training Epoch 8:  92%|█████████▏| 482/525 [13:18<00:47,  1.11s/it, loss=0.165]

In [ ]:
import torch
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [ ]:
def score_reranker(model, tokenizer, question, context, answer, max_len=256):
    model.eval()
    inputs = tokenizer(
        f"{question} [SEP] {context} [SEP] {answer}",
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_tensors="pt"
    )
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    with torch.no_grad():
        emb = model.encode(inputs['input_ids'], inputs['attention_mask'])
        score = model.reg_head(emb).squeeze(-1)
    return float(score.cpu().item())


In [ ]:
question = "أي من الصحابة ساعده النبي محمد (ﷺ) على التحرر من العبودية عن طريق زراعة أكثر من 300 نخلة؟"
answer = "سلمان الفارسي (رضي الله عنه)"
context = "في الإسلام يعتبر مفهوم البركة من المفاهيم العميقة والمهمة جدًا حيث يؤمن المسلمون أن البركة حين تنزل من عند الله فإنها تملأ الأمور العادية بالخيرات وتجلب النفع واليسر في شتى جوانب الحياة ويحرص المسلمون على بدء معظم أعمالهم بقول بسم الله طلبًا لبركة الله حتى أن العادات اليومية مثل الأكل أو النوم يمكن أن تتحول إلى أعمال عبادة عظيمة إذا نوى المسلم التقرب إلى الله بها وتذكره وفي التراث الإسلامي أمثلة كثيرة حول بركة الوقت والطعام والجهد وكيف أن الله سبحانه وتعالى يضاعف الأجر والثواب ويبارك في النيات الصالحة والأفعال الخيّرة."
context2 = "سلمان الفارسي رضي الله عنه هو أحد كبار صحابة النبي محمد صلى الله عليه وسلم وكان يبحث عن الحقيقة منذ شبابه فتنقل بين الأديان والمناطق حتى وصل إلى المدينة المنورة حيث التقى بالنبي فآمن به واعتنق الإسلام وأصبح من أحب الصحابة إلى قلب الرسول وقد ضرب مثلا في الحكمة والزهد والقوة في الإيمان وكان له دور بارز في غزوة الخندق عندما أشار بحفر الخندق حول المدينة وهو ما لم يكن معروفًا عند العرب آنذاك وقد قال عنه النبي صلى الله عليه وسلم عبارته المشهورة سلمان منا أهل البيت مما يدل على عظيم قدره ومنزلته في الإسلام وظل سلمان الفارسي يخدم الدين بكل إخلاص وعدل وتواضع حتى وفاته رضي الله عنه."
context3 = "سلمان الفارسي رضي الله عنه كان من كبار المستشارين للنبي محمد ﷺ وكان ذا عقل وحكمة بالغة فعندما حاصرت الأحزاب المدينة في غزوة الخندق أشار بحفر خندق حول المدينة وهو أسلوب لم تكن تعرفه العرب فأعجب النبي بفكرته ووافق عليها ونفذها المسلمون فكان سببا في صد العدو وانتصار المسلمين وقد اختلط نسبه العربي بالإسلامي فقال فيه النبي سلمان منا أهل البيت فكان ذلك شرفا عظيما له ومكانة عالية بين الصحابة"
context4 = "سلمان الفارسي رضي الله عنه كان قبل إسلامه يُلقب ابن النار لأنه كان خادما للنار بحسب الديانة الزرادشتية حيث كان والدُه يحرص عليه ويبقيه في البيت ويجعله يعتني بالنار المقدسة حتى أصابه حب البحث عن الحقيقة فمر بكنيسة النصارى وأعجبته صلاتهم وبدأت رحلة تنقله بين الديانات والمدن حتى التقى بالنبي محمد ﷺ وأسلم وقد اشتهر بعد ذلك بين الصحابة بلقب ابن الإسلام وقال عن نفسه أنا سلمان ابن الإسلام من بني آدم ليؤكد أن انتماءه الأول والأخير للإسلام وحده"
context5 = "سلمان الفارسي رضي الله عنه تحرر من العبودية بعد أن اشترط عليه مولاه زراعة ثلاثمئة نخلة ودفع أربعين أوقية من الذهب، فساعده الصحابة في جمع الفسائل وغرسها النبي صلى الله عليه وسلم بيده الشريفة فأثمرت كلها ببركة الله، ثم أهداه النبي قطعة ذهب أوفى بها الدين فنال حريته، وتُعد قصته مثالًا في الصبر والإيمان والتعاون على الخير."

pos_score = score_reranker(model, tokenizer, question, context, answer)
print("Score:", round(pos_score, 4))
pos_score = score_reranker(model, tokenizer, question, context2, answer)
print("Score:", round(pos_score, 4))
pos_score = score_reranker(model, tokenizer, question, context3, answer)
print("Score:", round(pos_score, 4))
pos_score = score_reranker(model, tokenizer, question, context4, answer)
print("Score:", round(pos_score, 4))
pos_score = score_reranker(model, tokenizer, question, context5, answer)
print("Score:", round(pos_score, 4))



Score: 0.0025
Score: 0.9327
Score: 0.8861
Score: 0.8016
Score: 0.9652
